In [1]:
pip install tidytext textblob

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 200)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm

from Help_Funs import count_chars, count_words, count_capital_chars, count_capital_words, count_sent, count_unique_words, count_stopwords, count_hashtags 

s3 = boto3.resource('s3')
s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'NLP-Disaster-Tweets/train.csv'
file_key_2 = 'NLP-Disaster-Tweets/test.csv'
file_key_3 = 'NLP-Disaster-Tweets/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
sample = pd.read_csv(file_content_stream_3)

In [6]:
## Defining stopwords
stop_words = set(stopwords.words('english'))
others = set(["1", "2", "it'll", "ill", "=", '+', "'s'", '"'])
stop_words = stop_words.union(others)

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stop_words]
    temp = " ".join(word for word in temp)
    return temp

In [7]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [9]:
train['clean_tweet'] = np.nan

for i in tqdm(range(0, train.shape[0])):
    
    train['clean_tweet'][i] =  clean_tweet(train['text'][i])

  0%|          | 0/7613 [00:00<?, ?it/s]/tmp/ipykernel_24983/4248643825.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['clean_tweet'][i] =  clean_tweet(train['text'][i])
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
100%|██████████| 7613/7613 [00:00<00:00, 38446.91it/s]


In [10]:
test['clean_tweet'] = np.nan

for i in tqdm(range(0, test.shape[0])):
    
    test['clean_tweet'][i] =  clean_tweet(test['text'][i])

  0%|          | 0/3263 [00:00<?, ?it/s]/tmp/ipykernel_24983/1846826793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['clean_tweet'][i] =  clean_tweet(test['text'][i])
100%|██████████| 3263/3263 [00:00<00:00, 23063.49it/s]


In [11]:
train['sentiment'] = train['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)
train['subjectivity'] = train['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

test['sentiment'] = test['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.polarity)
test['subjectivity'] = test['clean_tweet'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

In [13]:
train['sentiment_label'] = np.where(train['sentiment'] < 0, 'negative', 
                                    np.where(train['sentiment'] == 0, 'neutral', 'positive'))

test['sentiment_label'] = np.where(test['sentiment'] < 0, 'negative', 
                                   np.where(test['sentiment'] == 0, 'neutral', 'positive'))

In [16]:
train_dummies = pd.get_dummies(train['sentiment_label'])
train = pd.concat([train.drop(columns = ['sentiment', 'sentiment_label'], axis = 1), train_dummies], axis = 1)

test_dummies = pd.get_dummies(test['sentiment_label'])
test = pd.concat([test.drop(columns = ['sentiment', 'sentiment_label'], axis = 1), test_dummies], axis = 1)

In [19]:
train['char_count'] = train['clean_tweet'].apply(lambda x: count_chars(x))
train['word_count'] = train['clean_tweet'].apply(lambda x: count_words(x))
train['unique_word_count'] = train['clean_tweet'].apply(lambda x: count_unique_words(x))
                                                 
test['char_count'] = test['clean_tweet'].apply(lambda x: count_chars(x))
test['word_count'] = test['clean_tweet'].apply(lambda x: count_words(x))
test['unique_word_count'] = test['clean_tweet'].apply(lambda x: count_unique_words(x))
                                                 
## Average word length
train['avg_wordlength'] = train['char_count'] / train['word_count']
test['avg_wordlength'] = test['char_count'] / test['word_count']

## Unique words vs count words
train['unique_vs_words'] = train['unique_word_count'] / train['word_count']
test['unique_vs_words'] = test['unique_word_count'] / test['word_count']

In [22]:
train.to_csv('train.csv', index = False)
test.to_csv('test.csv', index = False)

In [18]:
train['target'].value_counts() / train.shape[0]

0    0.57034
1    0.42966
Name: target, dtype: float64

In [15]:
pd.crosstab(train['target'], train['sentiment_label'])

sentiment_label,negative,neutral,positive
target,,,
0,826,2014,1502
1,736,1707,828
